# In situ targeted base editing of gut bacteria in mice
## 16S analysis

© 2023 Eligo Bioscience

### Dependencies
* `pandas` 1.5.3
* `numpy` 1.24.2
* `seaborn` 0.12.2
* `matplotlib` 3.7.0

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import glob

In [2]:
TOP_N = 10

In [3]:
def parse_sample_id(df, sample_id_column="sample", suffix=""):
    df["timepoint" + suffix] = df[sample_id_column].str[:-3]
    df["mouse_id" + suffix] = df[sample_id_column].str[-3:]
    return df

In [4]:
def load_relab(path):
    relab = pd.read_csv(path, sep="\t")

    relab = (
        relab
        .drop("Tax_detail", axis=1) # Remove column that is not relab information
        .assign(D5M36=0) # Add dummy values for a missing sample
        .melt(id_vars="Taxonomy", var_name="sample", value_name="relab") 
    )
    
    parse_sample_id(relab)
    relab.timepoint = pd.Categorical(relab.timepoint, categories=['D4', 'D5', 'D6', 'D9', 'D12'], ordered=True)
    relab = relab.sort_values(["timepoint", "mouse_id"]).reset_index(drop=True) # Make sure days are always in the same order

    return(relab)

## Beta-diversity

In [5]:
paths = [
    "data/16S/beta-diversity/weighted_unifrac.tsv",
    "data/16S/beta-diversity/bray_curtis.tsv",
]

measure_names = [
    "Weighted UniFrac",
    "Bray-Curtis dissimilarity"
]

In [6]:
for path, mn in zip(paths, measure_names):
    dists = pd.read_csv(
                path, 
                sep="\t",
                index_col=0
            )

    melted_dist = (dists
                   .melt(var_name="sample_a", value_name="distance", ignore_index=False)
                   .reset_index()
                   .rename({"index": "sample_b"}, axis=1)
                  )
    parse_sample_id(melted_dist, "sample_a", suffix="_a")
    parse_sample_id(melted_dist, "sample_b", suffix="_b")

    melted_dist["timepoint_pair"] = melted_dist.timepoint_a + "-" + melted_dist.timepoint_b
    
    #===============
    
    g = sns.catplot(
        data=melted_dist[
            (melted_dist.mouse_id_a == melted_dist.mouse_id_b) & 
            (melted_dist.timepoint_pair.isin(["D4-D5", "D5-D6", "D5-D9", "D5-D12"]))
        ],
        x="timepoint_pair",
        y="distance",
        kind="box",
        height=3
    )
    g.set_ylabels(mn, fontsize=12)
    g.set_xlabels("Timepoints", fontsize=12)
    
    plt.savefig(f"plots/16S/{mn.replace(' ', '_').lower()}_box.svg", bbox_inches="tight")
    plt.savefig(f"plots/16S/{mn.replace(' ', '_').lower()}_box.pdf", bbox_inches="tight")
    plt.close()
    
    #===============
    
    g = sns.relplot(
        data=melted_dist[
            (melted_dist.mouse_id_a == melted_dist.mouse_id_b) & 
            (melted_dist.timepoint_pair.isin(["D4-D5", "D5-D6", "D5-D9", "D5-D12"]))
        ],
        x="timepoint_pair",
        y="distance",
        hue="mouse_id_a",
        style="mouse_id_a",
        markers=True,
        dashes=False,
        estimator=None,
        kind="line",
        alpha=0.7,
    )
    g.set_ylabels(mn)
    g.set_xlabels("Timepoints")

    g._legend.set_title("Mouse ID")
    
    plt.savefig(f"plots/16S/{mn.replace(' ', '_').lower()}_line.svg", bbox_inches="tight")
    plt.savefig(f"plots/16S/{mn.replace(' ', '_').lower()}_line.pdf", bbox_inches="tight")
    plt.close()

## Family-level bar chart

In [7]:
relab = load_relab("data/16S/composition/featureTable.sample.f.relative.tsv")
top_taxa = (
    relab
    .groupby("Taxonomy")
    .relab
    .sum()
    .sort_values(ascending=False)
    .drop("Others")
    .index[:TOP_N]
)


In [8]:
mean_relab_per_day = (
    relab
    [relab["sample"] != "D5M36"]
    .groupby(["Taxonomy", "timepoint"])
    .relab
    .mean()
    .reset_index()
    .sort_values(["timepoint", "relab"])
)

In [9]:
timepoints = relab.timepoint.cat.categories
mice = relab.mouse_id.unique()

fig, axs = plt.subplots(ncols=3, nrows=4, figsize=(7,10), sharex=True, sharey=True, layout="constrained")

for i, (mouse_id, ax) in enumerate(zip(mice, axs.flatten())):
    mouse_relab = relab[relab.mouse_id == mouse_id]
    bottoms = np.zeros(len(timepoints))

    for taxon in top_taxa:
        values = mouse_relab.loc[mouse_relab.Taxonomy == taxon, "relab"].to_numpy()
        ax.bar(timepoints, values, width=0.9, bottom=bottoms, label=taxon)
        bottoms += values


    value_others = 1-bottoms
    if mouse_id == "M36":
        value_others[1] = 0 # Missing timepoint

    ax.bar(timepoints, value_others, width=0.9, bottom=bottoms, label="Others", color="pink")        

    ax.set_title(mouse_id, fontsize=9)
    ax.tick_params(
        left=(i % 3 == 0),
        bottom=False
    )

    if (i == 3):
        ax.set_ylabel("Relative abundance")
        

# Plot the average
ax = axs[3,2]
bottoms = np.zeros(len(timepoints))

for taxon in top_taxa:
    values = mean_relab_per_day.loc[mean_relab_per_day.Taxonomy == taxon, "relab"].to_numpy()
    ax.bar(timepoints, values, width=0.9, bottom=bottoms, label=taxon)
    bottoms += values

value_others = 1-bottoms
ax.bar(timepoints, value_others, width=0.9, bottom=bottoms, label="Others", color="pink")
ax.tick_params(
    left=False,
    bottom=False
)

ax.set_title("Mean abundance per day", fontsize=9)

handles, labels = ax.get_legend_handles_labels()
axs[3,1].legend(handles, labels, loc='center', title="Families", prop={'size': 9})
axs[3,1].set_axis_off()

plt.savefig(f"plots/16S/daily_family_relab.pdf", bbox_inches="tight")
plt.savefig(f"plots/16S/daily_family_relab.svg", bbox_inches="tight")
plt.close()

## Family-level boxplot

In [10]:
family_relab = load_relab("data/16S/composition/featureTable.sample.f.relative.tsv")
family_relab = family_relab[
    (family_relab["sample"] != "D5M36") # Failed sample
]

In [11]:
top_families = (
    family_relab
    [family_relab.Taxonomy != "Others"]
    .groupby("Taxonomy")
    .relab
    .median() # This time sorting by median
    .sort_values(ascending=False)
    .iloc[:TOP_N]
    .index
)

In [12]:
top_families_relab = family_relab[family_relab.Taxonomy.isin(top_families)].copy()

relab_others = (1 - top_families_relab.groupby("sample").relab.sum())

relab_others_df = relab_others.reset_index()
relab_others_df["Taxonomy"] = "Others"
relab_others_df["timepoint"] = relab_others_df["sample"].str[:-3]
relab_others_df["mouse_id"] = relab_others_df["sample"].str[-3:]


relab_for_plotting = pd.concat([top_families_relab, relab_others_df])

assert (relab_for_plotting.groupby("sample").relab.sum() == 1).all()

In [13]:
relab_for_plotting["mean"] = relab_for_plotting.groupby("Taxonomy").relab.transform("mean")
relab_for_plotting["median"] = relab_for_plotting.groupby("Taxonomy").relab.transform("median")

In [14]:
sns.set_theme(
    context='paper', 
    style='ticks', 
    font_scale=1.2, 
    font='sans-serif', 
    palette="tab10", 
    rc={"lines.linewidth": 1.5, "lines.markersize": 4}
)

fig, ax = plt.subplots(figsize=(5,3))


sns.boxplot(
    data=relab_for_plotting[relab_for_plotting.timepoint.isin(("D4", "D12"))],
    x="Taxonomy",
    y="relab",
    hue="timepoint",
    width=0.6,
    fliersize=4,
    ax=ax
)

ax.set_yscale("log")
ax.set_ylabel("Relative abundance")
ax.set_xlabel(None)
ax.get_legend().get_title().set_text("Timepoint")
ax.set_ylim(ax.get_ylim()[0], 1)
    
ax.set_xticks(ax.get_xticks(), ax.get_xticklabels(), rotation=30, ha="right", rotation_mode="anchor")

for i in range(TOP_N):
    ax.axvline(i + .5, linestyle="--", c=".8", linewidth=1)
sns.despine()

fig.savefig(f"plots/16S/top_families_box.svg", bbox_inches="tight")
fig.savefig(f"plots/16S/top_families_box.pdf", bbox_inches="tight")
plt.close()

## E. coli abundance

In [15]:
species_relab = load_relab("data/16S/composition/featureTable.sample.s.relative.tsv")

species_relab = species_relab[
    (species_relab["sample"] != "D5M36") # Failed sample
]

In [16]:
sns.set_theme(
    context='paper', 
    style='ticks', 
    font_scale=1.5, 
    font='sans-serif', 
    palette="tab10", 
    rc={"lines.linewidth": 1.5, "lines.markersize": 6}
)

fig, ax = plt.subplots(figsize=(6,5.5), sharey=True)

ecoli_relab = species_relab[
    (species_relab.Taxonomy == "Escherichia_coli")
]

plot_kws = dict(
    x="timepoint",
    y="relab",
    hue="mouse_id",
    alpha=0.8,
)

sns.lineplot(
    data=ecoli_relab,
    ax=ax,
    legend=False,
    **plot_kws
)

sns.scatterplot(
    data=ecoli_relab,
    ax=ax,
    **plot_kws
)


ax.set(
    yscale="log",
    xlabel="Timepoint",
    ylabel="Relative abundance"
)

h,l = ax.get_legend_handles_labels()
ax.legend_.remove()
ax.legend(h,l, ncol=2, title="Mouse ID", loc="lower left", frameon=False)
    
sns.despine()

fig.savefig(f"plots/16S/ecoli.svg", bbox_inches="tight")
fig.savefig(f"plots/16S/ecoli.pdf", bbox_inches="tight")

plt.close()